# AG002 - Lucas Fajardo de Mello (GES - 171) e Arthur de Paula Assis (GES - 122)

Para começarmos a AG primeiramente temos que carregar o nosso dataset, para isso vamos utilizar a biblioteca pandas, com a função read_csv.

Vamos também importar algumas bibliotecas que utilizaremos depois.


In [36]:
import pandas as pd

dataset = pd.read_csv("Dataset/Wholesale customers data.csv")
dataset.head()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen
0,Retail,Other,12669,9656,7561,214,2674,1338
1,Retail,Other,7057,9810,9568,1762,3293,1776
2,Retail,Other,6353,8808,7684,2405,3516,7844
3,HoReCa,Other,13265,1196,4221,6404,507,1788
4,Retail,Other,22615,5410,7198,3915,1777,5185


## Ex 3

Nesse exercício nós trocamos os "rótulos" de Canal e Região por valores inteiros, facilitando futuros processos ao manejar o dataset.

In [37]:
dataset.Channel.replace({"HoReCa":0,"Retail":1}, inplace = True) #Altera os valores HoReCa e Retail para 0 e 1

dataset.Region.replace({"Lisbon":0,"Oporto":1,"Other":2}, inplace = True) #Altera os valores Lisbon, Oporto e Other para 0, 1 e 2
dataset.head()

C:\Users\ArthurPC\AppData\Local\Temp\ipykernel_7036\2407502995.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset.Channel.replace({"HoReCa":0,"Retail":1}, inplace = True) #Altera os valores HoReCa e Retail para 0 e 1
C:\Users\ArthurPC\AppData\Local\Temp\ipykernel_7036\2407502995.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option(

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen
0,1,2,12669,9656,7561,214,2674,1338
1,1,2,7057,9810,9568,1762,3293,1776
2,1,2,6353,8808,7684,2405,3516,7844
3,0,2,13265,1196,4221,6404,507,1788
4,1,2,22615,5410,7198,3915,1777,5185


## Ex 4

Nesse exercício iremos mudar a ordem das colunas do nosso dataframe, para isso iremos utilizar um artifício onde ao chamarmos um dataframe com uma lista das colunas desejadas em uma certa ordem o pandas retorna um dataframe com as colunas nessa mesma ordem, assim podemos atribuir esse dataframe reordenado ao original, assim o reorganizando sem a necessidade de chamadas de função mais complexas

In [38]:
novaOrdem =['Region', 'Fresh', 'Milk', 'Grocery', 'Frozen', 'Detergents_Paper', 'Delicatessen', 'Channel'] #Estabelecemos um array um com os valores na ordem nova dessejada

dataset = dataset [novaOrdem] #Altera a ordem para a nova ordem desejada

dataset.head()

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen,Channel
0,2,12669,9656,7561,214,2674,1338,1
1,2,7057,9810,9568,1762,3293,1776,1
2,2,6353,8808,7684,2405,3516,7844,1
3,2,13265,1196,4221,6404,507,1788,0
4,2,22615,5410,7198,3915,1777,5185,1


## Ex 5

Nesse exercício iremos separar nosso dataset em uma seção de teste e outra para treinos do nosso modelo de ML, iremos utilizar um split de 80/20.

Para realizar essa divisão utilizaremos a função train_test_split do sklearn

In [39]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset, test_size=0.2)

## Ex 6 

Nesse exercício iremos escolher um modelo de ML, decidimos por escolher o modelo Naive Bayes, por ser um modelo simples baseado em matemática e ser amplamente utilizado para predições similares no contexto de varejo.

Antes de treinar o nosso modelo, também vamos utilizar um algoritmo de Forward Selection com K-fold cross validation para escolher apenas as features que terão um impacto significativo no modelo.

In [40]:
from sklearn.model_selection import cross_val_score

def K_fold_validation_scoring(model,X,y,scoring:str):
    scores = cross_val_score(model, X, y, cv=10,scoring=scoring)#Faz 10-fold cross validation e retorna as pontuações de NRMSE
    return scores.mean()#retorna a média de pontuação do NRMSE

In [41]:
def fwd_selection_cross_validation(X,y,features,minScoreVariation : float,detailedOutput : bool,scoring : str,model):
    used_features=[]#lista para armazenar as features que devem ser usadas no modelo, de acordo com o forward selection 
    unused_features = list(features)#inicialmente lista todas as features disponíveis para o modelo, porém, ao final, conterá as features não usadas
    current_score = float('-inf')#variável para armazenar a pontuação do modelo atual
    best_current_score = float('-inf')#variável para armazenar a melhor pontuação registrada

    while current_score == best_current_score and unused_features:
        score_for_feature = []#lista para armazenar a pontuação obtida com cada feature
        for feature in unused_features:#faz uma iteração para cada feature não utilizada, além de realizar o 10-fold cross validation para cada uma
            current_features = used_features + [feature]
            model.fit(X=X[current_features],y=y)
            score = K_fold_validation_scoring(model,X[current_features],y,scoring)
            score_for_feature.append((score,feature))
        
        score_for_feature.sort()#ordena as pontuações de forma que o menor valor fique no início da lista
        best_current_score,best_feature = score_for_feature.pop(score_for_feature.__len__()-1)#obtém a melhor feature junto com sua pontuação

        if(detailedOutput):#se detailedOutput for True, exibe informações mais detalhadas sobre o processo
            print('Best current score: '+str(best_current_score)+'\nWith addition of feature: '+str(best_feature))

        if current_score < (best_current_score-minScoreVariation):#se a nova pontuação - a variação mínima definida pelo usuário
            unused_features.remove(best_feature)#for menor que a pontuação anterior, adiciona a feature às features selecionadas
            used_features.append(best_feature)#e substitui a pontuação atual pela nova pontuação
            current_score = best_current_score
        elif(detailedOutput):
                print('Rejected feature: '+best_feature+'\nReason : Low score increase (<'+str(minScoreVariation)+')\nFinal model has '+str(used_features.__len__())+' features')

    return used_features,current_score

# O código de ambas as funções foram feitos por mim, Lucas Fajardo, para um projeto durante meu intercâmbio
# Já os comentários anteriormente estavam em inglês por culpa disso, então foram traduzidos por mim, Arthur Assis

In [42]:
from time import time
from sklearn.naive_bayes import GaussianNB

features = train_data.columns.drop("Channel") #Seleciona apenas a feature a serem utilizadas pelo modelo

start = time() # Tempo inicial
selected_features,acc = fwd_selection_cross_validation(X=train_data[features],y=train_data['Channel'],features=features,minScoreVariation=0.01,detailedOutput=True,scoring = 'accuracy',model= GaussianNB())
elapsedTime = time()-start # Tempo total

print('Selected features: '+str(selected_features)+'\nAccuracy: '+str(acc)+'\nIn '+str(elapsedTime)+' Seconds')

Best current score: 0.9063492063492063
With addition of feature: Detergents_Paper
Best current score: 0.9092063492063491
With addition of feature: Region
Rejected feature: Region
Reason : Low score increase (<0.01)
Final model has 1 features
Selected features: ['Detergents_Paper']
Accuracy: 0.9063492063492063
In 1.166161298751831 Seconds


O nosso modelo está apresentando uma precisão de 90.63% apenas com a feature Detergents_Paper, isso é altamente incomum, e mais pra frente com mais iremos analisar isso mais a fundo com outras métricas

## EX 7

Agora iremos testar a precisão do nosso modelo com os nossos dados de teste.

In [43]:
NBmodel = GaussianNB()

NBmodel.fit(train_data[selected_features],y=train_data['Channel'])

Predictions = NBmodel.predict(test_data[selected_features])
Predictions[:10]

array([0, 1, 0, 0, 1, 1, 1, 0, 0, 0], dtype=int64)

Vamos também treinar um modelo com todas as features disponíveis para ver a diferença nas métricas

In [44]:
NBmodelFull = GaussianNB()

NBmodelFull.fit(train_data[features],y=train_data['Channel'])

PredictionsFull = NBmodelFull.predict(test_data[features])
PredictionsFull[:10]

array([0, 1, 0, 0, 1, 1, 1, 0, 0, 0], dtype=int64)

## Ex 8

Nesse exercício iremos avaliar os nossos modelos com base em diversas métricas, a maioria obtida através da função classification_report além da acurácia dos modelos e a confusion matrix

In [45]:
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix

print(classification_report(test_data["Channel"],Predictions,target_names=["HoReCa","Retail"]))
testAcc = accuracy_score(test_data['Channel'], Predictions)
print('Accuracy: '+str(testAcc))
confusionMat = confusion_matrix(test_data['Channel'], Predictions)
print('Confusion matrix:\n')
print(confusionMat)

              precision    recall  f1-score   support

      HoReCa       0.91      0.93      0.92        54
      Retail       0.88      0.85      0.87        34

    accuracy                           0.90        88
   macro avg       0.89      0.89      0.89        88
weighted avg       0.90      0.90      0.90        88

Accuracy: 0.8977272727272727
Confusion matrix:

[[50  4]
 [ 5 29]]


No geral obtivemos boas métricas, sempre com uma precisão de aproximadamente 90%, sendo excelente para um modelo tão simples quanto o Naive Bayes, ainda mais com apenas uma feature, além de recall e F1 próximos de 1 nos indicando que o nosso modelo não conseguiu métricas boas apenas "chutando" o mesmo valor toda vez, informação que conseguimos comprovar através da Confusion Matrix

In [46]:
print(classification_report(test_data["Channel"],PredictionsFull,target_names=["HoReCa","Retail"]))
testAcc = accuracy_score(test_data['Channel'], PredictionsFull)
print('Accuracy: '+str(testAcc))
confusionMat = confusion_matrix(test_data['Channel'], PredictionsFull)
print('Confusion matrix:\n')
print(confusionMat)

              precision    recall  f1-score   support

      HoReCa       0.93      0.93      0.93        54
      Retail       0.88      0.88      0.88        34

    accuracy                           0.91        88
   macro avg       0.90      0.90      0.90        88
weighted avg       0.91      0.91      0.91        88

Accuracy: 0.9090909090909091
Confusion matrix:

[[50  4]
 [ 4 30]]


Podemos ver que as métricas obtidas utilizando um modelo com todas as features foram as mesmas das obtidas com um modelo utilizando apenas Detergents_Paper, ou seja conseguimos resultados iguais com um modelo muito mais simples, assim economizando tempo e poder computacional.

## Ex 9

Nesse exercício o usuário deve ser capaz de entrar com valores para que o modelo preveja qual o canal de vendas os dados pertencem.

In [47]:
prediction = NBmodel.predict([[int(input("Insira o valor de Detergents_Paper: "))]])
if prediction[0] == 0:
    print("Origem prevista é HoReCa")
else:
    print("Origem prevista é Retail")

Origem prevista é HoReCa


C:\Users\ArthurPC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


Vamos também fazer uma versão onde utilizamos o modelo completamente

In [ ]:
input_features = []
input_features.append(int(input("Entre com o valor de Region :")))
input_features.append(int(input("Entre com o valor de Fresh :")))
input_features.append(int(input("Entre com o valor de Milk :")))
input_features.append(int(input("Entre com o valor de Grocery :")))
input_features.append(int(input("Entre com o valor de Frozen :")))
input_features.append(int(input("Entre com o valor de Detergents_Paper :")))
input_features.append(int(input("Entre com o valor de Delicatessen :")))

prediction = NBmodelFull.predict([input_features])
if prediction[0] == 0:
    print("Origem prevista é HoReCa")
else:
    print("Origem prevista é Retail")

Origem prevista é HoTeCa


C:\Users\ArthurPC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
